# Coursera's Courses Chatbot

##### This notebook contians the process of building a chatbot with langchain library, using Google's Gemini as an LLM and Pinecone as a vector database

## Importing the libraries

In [41]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate, MessagesPlaceholder
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone as pc
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain, create_history_aware_retriever, RetrievalQA
from langchain.chains.question_answering import load_qa_chain
from langchain.schema import *

from dotenv import find_dotenv, load_dotenv
import os

## Calling APIs

In [42]:
load_dotenv()

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')

## Calling the vector database

In [43]:
pinecone_environment = 'gcp-starter'
# Loading Pinecone
pc = pc()
index = pc.Index("mychatpot")

## Function for getting the answer for a query

In [75]:
def get_answer(query):

    model = 'models/embedding-001'
    embed = GoogleGenerativeAIEmbeddings(model=model)

    vstore = Pinecone(embedding=embed, index=index, text_key='text')
    retriever = vstore.as_retriever(k=10)

    llm = ChatGoogleGenerativeAI(model='gemini-pro', temperature=0.7 ,convert_system_message_to_human=True)

    template = """You play the role of an assistant who will help students to find courses suited to them.
A student is going to ask you a question, your task is to answer the question using the context.
If the question is not related to the context, guide the student to ask you about Coursera's courses.\n\n
Context: \n {context}?\n
Question: \n {question} \n
Answer:"""

    QA_CHIAN_PROMPT = PromptTemplate.from_template(template)

    qa_chian = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type_kwargs={
            'prompt':QA_CHIAN_PROMPT
        }
    )
    response = qa_chian.invoke(query)
    return response['result']

In [78]:
get_answer('give me some cooking courses and their ratings')

'1. The Science of Gastronomy, rating: 4.6\n2. More Chinese for Beginners, rating: 4.6'